### PROJECT : TEXT TO SQL via PROMPT ENGINEERING


In [ ]:
from google.colab import drive
drive.mount("/content/data")

Drive already mounted at /content/data; to attempt to forcibly remount, call drive.mount("/content/data", force_remount=True).


In [ ]:
import sqlite3
import pandas as pd
import os

In [ ]:
employee_schema = """
CREATE TABLE IF NOT EXISTS employees (
    employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    phone_number VARCHAR(50),
    hire_date DATE,
    job_title VARCHAR(50),
    department VARCHAR(50)
);
"""
salary_schema = """
CREATE TABLE IF NOT EXISTS salaries (
    salary_id INTEGER PRIMARY KEY AUTOINCREMENT,
    employee_id INTEGER NOT NULL,
    salary INTEGER NOT NULL,
    bonus INTEGER DEFAULT 0,
    pay_date DATE NOT NULL,
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE
);
"""

In [ ]:
db_name = 'employee.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

In [67]:
COLUMN_DATA_TYPES = {
    'employees': {
        'employee_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'email': 'object',
        'phone_number': 'object',
        'hire_date': 'datetime64[ns]',
        'job_title': 'object',
        'department': 'object'
    },
    'salaries': {
        'salary_id': 'int64',
        'employee_id': 'int64',
        'salary': 'int64',
        'bonus': 'int64',
        'pay_date': 'datetime64[ns]'
    }
}

# --- Database setup ---
db_name = 'employee.db'
conn = None  # Initialize connection to None

try:

    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys = ON;")
    print(f"Database '{db_name}' created and connected successfully.")

    cursor.execute(employee_schema)
    cursor.execute(salary_schema)
    print("Tables 'Employee' and 'Salary' created successfully.")

    csv_to_table_map = {
        '/content/data/MyDrive/Datasets/EMPLOYEE_DATA.csv': 'employees',
        '/content/data/MyDrive/Datasets/SALARY_DATA.csv': 'salaries'
    }

    def get_existing_employee_ids(conn):
        query = "SELECT employee_id FROM employees"
        return set(pd.read_sql(query, conn)['employee_id'])

    for csv_file, table_name in csv_to_table_map.items():

        if not os.path.exists(csv_file):
            print(f" Warning: '{csv_file}' not found. Skipping '{table_name}'.")
            continue

        print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

        # Read CSV
        df = pd.read_csv(csv_file)

        # 1. Expected schema
        expected_schema = COLUMN_DATA_TYPES[table_name]
        expected_cols = list(expected_schema.keys())

        # 2. Drop extra columns
        df = df[df.columns.intersection(expected_cols)]

        # 3. Add missing columns
        for col in expected_cols:
            if col not in df.columns:
                df[col] = None

        # 4. Reorder columns
        df = df[expected_cols]

        # 5. Enforce data types
        for col, dtype in expected_schema.items():
            if 'datetime' in dtype:
                df[col] = pd.to_datetime(
                    df[col], format="%d/%m/%Y", errors='coerce'
                )
            else:
                try:
                    df[col] = df[col].astype(dtype)
                except (ValueError, TypeError) as e:
                    print(
                        f"  - Warning: Could not convert '{col}' to {dtype}. "
                        f"Error: {e}"
                    )

        if table_name == 'employees':
            df = df.drop(columns=['employee_id'], errors='ignore')

        elif table_name == 'salaries':
            df = df.drop(columns=['salary_id'], errors='ignore')

            valid_employee_ids = get_existing_employee_ids(conn)
            initial_rows = len(df)

            df = df[df['employee_id'].isin(valid_employee_ids)]

            dropped = initial_rows - len(df)
            if dropped > 0:
                print(
                    f"  Foreign key validation: Dropped {dropped} "
                    f"salary record(s) with invalid employee_id."
                )

            if df.empty:
                print("  No valid salary records left. Skipping insert.")
                continue

        df.to_sql(
            table_name,
            conn,
            if_exists='append',
            index=False
        )

        print(f"  -> Data from '{csv_file}' loaded into '{table_name}' successfully.")
    conn.commit()
    print("\n✅Data committed to the database successfully.")

except sqlite3.Error as e:
    print(f" Database error: {e}")

except pd.errors.EmptyDataError as e:
    print(f" Pandas error: {e}. CSV file might be empty.")

except KeyError as e:
    print(f" Schema error: Missing column definition for {e}")

except Exception as e:
    print(f" Unexpected error: {e}")

finally:
    if conn:
        conn.close()
        print(" Database connection closed.")

Database 'employee.db' created and connected successfully.
Tables 'Employee' and 'Salary' created successfully.

Processing '/content/data/MyDrive/Datasets/EMPLOYEE_DATA.csv' for table 'employees'...
  -> Data from '/content/data/MyDrive/Datasets/EMPLOYEE_DATA.csv' loaded into 'employees' successfully.

Processing '/content/data/MyDrive/Datasets/SALARY_DATA.csv' for table 'salaries'...
  -> Data from '/content/data/MyDrive/Datasets/SALARY_DATA.csv' loaded into 'salaries' successfully.

✅Data committed to the database successfully.
 Database connection closed.


In [ ]:
!pip install google-generativeai


In [ ]:
from google.colab import userdata
import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"

In [68]:
prompt = """

###ROLE###
You are a highly skilled Text-to-SQL translator with expertise in SQL syntax, database schema interpretation, and natural language understanding. You generate syntactically correct and semantically accurate SQL queries based on user input and a given database schema.

###CONTEXT###
The user is working with a relational database for an employee platform. The database includes two main tables: `employees` and `salaries`. The goal is to allow users to input natural language queries (in English), and have the model return equivalent SQL statements that accurately extract the requested data using the given schema.

Here is the full schema:

**Employees Table**
```sql
CREATE TABLE IF NOT EXISTS employees (
    employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    phone_number VARCHAR(50),
    hire_date DATE,
    job_title VARCHAR(50),
    department VARCHAR(50)
);
````

**Salaries Table**

```sql
CREATE TABLE IF NOT EXISTS salaries (
    salary_id INTEGER PRIMARY KEY AUTOINCREMENT,
    employee_id INTEGER NOT NULL,
    salary INTEGER NOT NULL,
    bonus INTEGER DEFAULT 0,
    pay_date DATE NOT NULL,
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE
);
```

###TASK###
1.Read a natural language query about the employee data.
2.Interpret the user's intent based on the schema provided.
3.Generate a valid SQLite SELECT query that returns the expected result.
4.Ensure correct table joins, column selection, filtering, and grouping as necessary.
5.Handle aggregate functions (COUNT, AVG, SUM) correctly.
6.Handle date filtering using SQLite functions like strftime('%Y', column) instead of MySQL-style YEAR() or MONTH().
7.Handle current date references using date('now') instead of NOW() or CURRENT_DATE.
8.Use proper LIMIT syntax supported in SQLite.
9.Only return valid SQLite queries — no explanations or extra text.

###CONSTRAINTS###

*Only return a valid SQLite SQL query as output — no explanations or extra text.
*Do not create or modify tables.
*Do not assume the existence of tables or columns not provided in the schema.
*Avoid subqueries unless absolutely necessary.
*Prefer readability: indent joins and clauses properly.

###EXAMPLES###
**Input:**
"Show the names and email addresses of employees working in the IT department."

**output:**

```sql
SELECT first_name, last_name, email
FROM employees
WHERE department = 'IT';
```

**Input:**
"List the total salary paid to each department."
**Output:**

```sql
SELECT department, SUM(salary) AS total_salary
FROM employees
JOIN salaries ON employees.employee_id = salaries.employee_id
```

**Input:** "List all employees along with their net salary for the most recent pay date"
**Output:**

```sql
SELECT employees.first_name, employees.last_name, salaries.salary + salaries.bonus AS net_salary
FROM employees
```

**Input:** "Show me the employees average salary in 2023"
**Output:**

```sql
SELECT AVG(salary) AS average_salary
FROM salaries
WHERE strftime('%Y', pay_date) = '2023';
```

###OUTPUT FORMAT###
Return only the SQLite SQL query as a code block using triple backticks and the sql language tag, like this:

```sql
-- Your SQL query here
```
"""

In [69]:
from openai import OpenAI
client = OpenAI()

def get_sql_query_via_openrouter(prompt, user_query):
    response = client.chat.completions.create(
        model="qwen/qwen-2.5-7b-instruct",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_query}
        ],
        temperature=0
    )

    sql = (
        response.choices[0].message.content
        .replace("```sql", "")
        .replace("```", "")
        .strip()
    )

    return sql

In [70]:
import sqlite3
import pandas as pd

def execute_query(query, db_name="employee.db"):
    conn = None
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]

        df = pd.DataFrame(results, columns=columns)
        return df

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None
    finally:
        if conn:
            conn.close()


In [74]:
def text2sql(prompt, user_query):
    sql_query = get_sql_query_via_openrouter(prompt, user_query)
    return execute_query(sql_query)



In [72]:
text2sql(prompt, "Show me the employees average salary in 2023")


Executing query on 'employee.db':
SELECT AVG(salary) AS average_salary
FROM salaries
WHERE strftime('%Y', pay_date) = '2023';


,average_salary
0,117233.941176


In [75]:
text2sql(prompt,"List the top 5 highest paid employees")


Executing query on 'employee.db':
SELECT first_name, last_name, salary
FROM employees
JOIN salaries ON employees.employee_id = salaries.employee_id
ORDER BY salary DESC
LIMIT 5;


,first_name,last_name,salary
0,Ursala,Wapplington,199509
1,Ursala,Wapplington,199509
2,Ursala,Wapplington,199509
3,Ursala,Wapplington,199509
4,Roth,Semrad,199153


In [76]:
text2sql(prompt,"List the total salary paid to each department")


Executing query on 'employee.db':
SELECT department, SUM(salary) AS total_salary
FROM employees
JOIN salaries ON employees.employee_id = salaries.employee_id
GROUP BY department;


,department,total_salary
0,Engineering,14281104
1,Finance,19782452
2,HR,18470540
3,Marketing,28053448
4,Sales,14645312


In [77]:
text2sql(prompt,"List all employees along with their net salary for the most recent pay date")


Executing query on 'employee.db':
SELECT employees.first_name, employees.last_name, 
       MAX(salaries.pay_date) AS latest_pay_date, 
       salaries.salary + salaries.bonus AS net_salary
FROM employees
JOIN salaries ON employees.employee_id = salaries.employee_id
GROUP BY employees.employee_id, employees.first_name, employees.last_name
ORDER BY latest_pay_date DESC;


,first_name,last_name,latest_pay_date,net_salary
0,Dorry,Kempster,2025-12-30 00:00:00,71704
1,Kliment,Langstaff,2025-12-17 00:00:00,67040
2,Toma,Kilbee,2025-12-05 00:00:00,155365
3,Ricky,Newing,2025-12-05 00:00:00,190117
4,Joell,Eastbrook,2025-11-17 00:00:00,208821
...,...,...,...,...
118,Penelopa,Desquesnes,2023-03-16 00:00:00,103021
119,Reggy,Dulling,2023-02-17 00:00:00,199722
120,Eugine,Ausher,2023-01-28 00:00:00,162537
121,Herminia,Leggitt,2023-01-05 00:00:00,79878
